In [5]:
# !pip install tabulate
import pandas as pd
import re
from collections import defaultdict

In [6]:
df = pd.read_csv("songs_soundchart.csv")
df.head()

,Song,Song uuid,Song url,Song isrc,ISRC Country,Release date,Song genres,Song sub genres,Label,Label type,...,Boomplay favorites Change %,Genius views Total,Genius views Change,Genius views Change %,Melon likes Total,Melon likes Change,Melon likes Change %,Audiomack plays Total,Audiomack plays Change,Audiomack plays Change %
0,Blinding Lights,b6ac0f7e-112b-11ea-8a46-a81e84f2a475,https://app.soundcharts.com/app/song/b6ac0f7e-...,USUG11904206,United States,2019-11-29,R&B,"R&B, R&B, Funk & Soul","Republic, XO","Universal, Indie",...,NaN,2968511.0,9275.0,0.31,117927.0,613.0,0.52,NaN,NaN,NaN
1,blinding lights,1fd739d9-54ef-4458-903c-532afcfc9d06,https://app.soundcharts.com/app/song/1fd739d9-...,NL9KR2454563,Netherlands,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Blinding Lights,b86fc021-a5ed-4ae7-bf60-93ec1240260d,https://app.soundcharts.com/app/song/b86fc021-...,QZLBC2302779,United States,2022-06-24,Pop,Pop,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Shape of You,11e826a3-6ed8-a3f2-b5be-a0369fe50396,https://app.soundcharts.com/app/song/11e826a3-...,GBAHS1600463,United Kingdom,2017-01-06,"Pop, Folk","Pop, Singer/Songwriter",Asylum,Warner,...,NaN,15431942.0,34522.0,0.22,347657.0,137.0,0.04,NaN,NaN,NaN
4,Someone You Loved,1fcf0080-e45d-11e8-ba0b-549f35141000,https://app.soundcharts.com/app/song/1fcf0080-...,DEUM71807062,Germany,2018-11-08,"Alternative, Rock","Alternative, Rock",Vertigo Berlin,Universal,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Used to speed up column selection
# column_names_list = df.columns.tolist()
# column_names_list

['Song',
 'Song uuid',
 'Song url',
 'Song isrc',
 'ISRC Country',
 'Release date',
 'Song genres',
 'Song sub genres',
 'Label',
 'Label type',
 'Artist(s)',
 'Artist Country',
 'Explicit',
 'Themes',
 'Moods',
 'People mention',
 'Cultural references',
 'Brands',
 'Narrative style',
 'Locations',
 'Emotional intensity',
 'Imagery',
 'Complexity',
 'Rhyme scheme',
 'Repetitiveness',
 'Spotify stream Total',
 'Spotify stream Change',
 'Spotify stream Change %',
 'Spotify popularity Total',
 'Spotify popularity Change',
 'Spotify popularity Change %',
 'Youtube views Total',
 'Youtube views Change',
 'Youtube views Change %',
 'Tiktok videos Total',
 'Tiktok videos Change',
 'Tiktok videos Change %',
 'Airplay plays Total',
 'Airplay plays Change',
 'Airplay plays Change %',
 'Shazam counts Total',
 'Shazam counts Change',
 'Shazam counts Change %',
 'Soundcloud plays Total',
 'Soundcloud plays Change',
 'Soundcloud plays Change %',
 'Line-music streams Total',
 'Line-music streams Change',
 'Line-music streams Change %',
 'Anghami plays Total',
 'Anghami plays Change',
 'Anghami plays Change %',
 'Gaana favorites Total',
 'Gaana favorites Change',
 'Gaana favorites Change %',
 'Jiosaavn plays Total',
 'Jiosaavn plays Change',
 'Jiosaavn plays Change %',
 'Boomplay favorites Total',
 'Boomplay favorites Change',
 'Boomplay favorites Change %',
 'Genius views Total',
 'Genius views Change',
 'Genius views Change %',
 'Melon likes Total',
 'Melon likes Change',
 'Melon likes Change %',
 'Audiomack plays Total',
 'Audiomack plays Change',
 'Audiomack plays Change %']

In [8]:
columns_to_keep = ['Song',
 'Song uuid',
 'Song url',
 'Song isrc',
 'ISRC Country',
 'Release date',
 'Song genres',
 'Artist(s)',
 'Artist Country',
 'Themes',
 'Spotify stream Total',
 'Spotify stream Change',
 'Spotify stream Change %',
 'Spotify popularity Total',
 'Spotify popularity Change',
 'Spotify popularity Change %',
 'Youtube views Total',
 'Tiktok videos Total',
 'Airplay plays Total',
 'Shazam counts Total']

# Filter the DataFrame to keep only the desired columns
filtered_df = df[columns_to_keep]


In [9]:
filtered_df.head(10)

,Song,Song uuid,Song url,Song isrc,ISRC Country,Release date,Song genres,Artist(s),Artist Country,Themes,Spotify stream Total,Spotify stream Change,Spotify stream Change %,Spotify popularity Total,Spotify popularity Change,Spotify popularity Change %,Youtube views Total,Tiktok videos Total,Airplay plays Total,Shazam counts Total
0,Blinding Lights,b6ac0f7e-112b-11ea-8a46-a81e84f2a475,https://app.soundcharts.com/app/song/b6ac0f7e-...,USUG11904206,United States,2019-11-29,R&B,The Weeknd,Canada,"Loneliness, Love, Heartbreak, Escapism",4673836529,45739583,0.99,87.0,1.0,1.16,9.144275e+08,2705250.0,154656.0,30027808.0
1,blinding lights,1fd739d9-54ef-4458-903c-532afcfc9d06,https://app.soundcharts.com/app/song/1fd739d9-...,NL9KR2454563,Netherlands,NaN,NaN,FireVRMusic,NaN,NaN,4673836529,44098962,0.95,19.0,-9.0,-32.14,NaN,NaN,NaN,NaN
2,Blinding Lights,b86fc021-a5ed-4ae7-bf60-93ec1240260d,https://app.soundcharts.com/app/song/b86fc021-...,QZLBC2302779,United States,2022-06-24,Pop,"Batusyex, imnotege, Lanceas, rufflws, xSyborg","n/a, Turkey, Turkey, Germany, Turkey",NaN,4673836529,44098962,0.95,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Shape of You,11e826a3-6ed8-a3f2-b5be-a0369fe50396,https://app.soundcharts.com/app/song/11e826a3-...,GBAHS1600463,United Kingdom,2017-01-06,"Pop, Folk",Ed Sheeran,United Kingdom,"Love, Relationships, Desire, Self-discovery",4193269875,33855890,0.81,85.0,0.0,0.00,7.967218e+09,1701532.0,82429.0,28875592.0
4,Someone You Loved,1fcf0080-e45d-11e8-ba0b-549f35141000,https://app.soundcharts.com/app/song/1fcf0080-...,DEUM71807062,Germany,2018-11-08,"Alternative, Rock",Lewis Capaldi,United Kingdom,"Heartbreak, Depression, Love, Loneliness, Grief",3770959284,36229005,0.97,68.0,2.0,3.03,6.937573e+08,1932891.0,54922.0,30360617.0
5,As It Was,39fdb751-d3a7-4b2d-a4b5-50075d4ca086,https://app.soundcharts.com/app/song/39fdb751-...,USSM12200612,United States,2022-03-31,Pop,Harry Styles,United Kingdom,"Loneliness, Heartbreak, Mental health, Relatio...",3731855719,38439574,1.04,86.0,0.0,0.00,1.199166e+09,2485.0,227634.0,NaN
6,Starboy,11e826b6-adac-861c-a40a-a0369fe50396,https://app.soundcharts.com/app/song/11e826b6-...,USUG11600976,United States,2016-09-22,"R&B, Electro","Daft Punk, The Weeknd","France, Canada","Ambition, Materialism, Success, Love, Loneliness",3704089716,49815366,1.36,87.0,0.0,0.00,3.133207e+09,208252.0,17346.0,16387360.0
7,Starboy,11e826b6-adf8-c900-a130-a0369fe50396,https://app.soundcharts.com/app/song/11e826b6-...,USUG11600977,United States,2016-11-24,"R&B, Electro","Daft Punk, The Weeknd","France, Canada","Ambition, Materialism, Success, Love, Loneliness",3704089716,49815366,1.36,56.0,0.0,0.00,2.585120e+06,NaN,48.0,16385428.0
8,Sunflower (Spider-Man: Into the Spider-Verse),8475156c-d6d7-11e8-a15a-525400009efb,https://app.soundcharts.com/app/song/8475156c-...,USUM71814888,United States,2018-10-18,"Hip Hop, Soundtrack","Post Malone, Swae Lee","United States, United States","Heartbreak, Love, Relationships, Loneliness",3683693906,33134358,0.91,80.0,-1.0,-1.23,5.002809e+08,734948.0,37975.0,2078338.0
9,Sweater Weather,11e82693-06c8-f70e-9b64-aa1c026db3d8,https://app.soundcharts.com/app/song/11e82693-...,USSM11300080,United States,2012-01-01,"Alternative, Rock",The Neighbourhood,United States,"Love, Relationships, Loneliness, Desire, Intimacy",3524300277,68700567,1.99,89.0,0.0,0.00,1.363604e+09,631230.0,7025.0,16707785.0


In [10]:
# This will drop rows where 'Song genres' has NaN values and return a new DataFrame
df_cleaned = filtered_df.dropna(subset=['Song genres']).copy()  # Had to use .copy() to avoid SettingWithCopyWarning
df_cleaned.head()

,Song,Song uuid,Song url,Song isrc,ISRC Country,Release date,Song genres,Artist(s),Artist Country,Themes,Spotify stream Total,Spotify stream Change,Spotify stream Change %,Spotify popularity Total,Spotify popularity Change,Spotify popularity Change %,Youtube views Total,Tiktok videos Total,Airplay plays Total,Shazam counts Total
0,Blinding Lights,b6ac0f7e-112b-11ea-8a46-a81e84f2a475,https://app.soundcharts.com/app/song/b6ac0f7e-...,USUG11904206,United States,2019-11-29,R&B,The Weeknd,Canada,"Loneliness, Love, Heartbreak, Escapism",4673836529,45739583,0.99,87.0,1.0,1.16,9.144275e+08,2705250.0,154656.0,30027808.0
2,Blinding Lights,b86fc021-a5ed-4ae7-bf60-93ec1240260d,https://app.soundcharts.com/app/song/b86fc021-...,QZLBC2302779,United States,2022-06-24,Pop,"Batusyex, imnotege, Lanceas, rufflws, xSyborg","n/a, Turkey, Turkey, Germany, Turkey",NaN,4673836529,44098962,0.95,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Shape of You,11e826a3-6ed8-a3f2-b5be-a0369fe50396,https://app.soundcharts.com/app/song/11e826a3-...,GBAHS1600463,United Kingdom,2017-01-06,"Pop, Folk",Ed Sheeran,United Kingdom,"Love, Relationships, Desire, Self-discovery",4193269875,33855890,0.81,85.0,0.0,0.00,7.967218e+09,1701532.0,82429.0,28875592.0
4,Someone You Loved,1fcf0080-e45d-11e8-ba0b-549f35141000,https://app.soundcharts.com/app/song/1fcf0080-...,DEUM71807062,Germany,2018-11-08,"Alternative, Rock",Lewis Capaldi,United Kingdom,"Heartbreak, Depression, Love, Loneliness, Grief",3770959284,36229005,0.97,68.0,2.0,3.03,6.937573e+08,1932891.0,54922.0,30360617.0
5,As It Was,39fdb751-d3a7-4b2d-a4b5-50075d4ca086,https://app.soundcharts.com/app/song/39fdb751-...,USSM12200612,United States,2022-03-31,Pop,Harry Styles,United Kingdom,"Loneliness, Heartbreak, Mental health, Relatio...",3731855719,38439574,1.04,86.0,0.0,0.00,1.199166e+09,2485.0,227634.0,NaN


In [11]:
# This was unique given the data in Release date. Just doing 1 .fillna would not work. We had to first fill NaN values with a string version of 1776 (Just to have a value of something), to keep the regex from breaking due to a int being where it was looking for a string.

# Fill NaN values in 'Release date' with 1776 and convert to string
df_cleaned['Year Released'] = df_cleaned['Release date'].fillna('1776').astype(str)

# Extract the year from 'Year Released' using regex
df_cleaned['Year Released'] = df_cleaned['Year Released'].str.extract(r'(\d{4})')

# Fill any remaining NaN values in 'Year Released' with 1776
df_cleaned['Year Released'] = df_cleaned['Year Released'].fillna(1776)

# Convert the 'Year Released' column to integers so further mathamatical processes can be done
df_cleaned['Year Released'] = df_cleaned['Year Released'].astype(int)

In [12]:
# Display the DataFrame with the extracted year
print(df_cleaned[['Release date', 'Year Released']])

      Release date  Year Released
0       2019-11-29           2019
2       2022-06-24           2022
3       2017-01-06           2017
4       2018-11-08           2018
5       2022-03-31           2022
...            ...            ...
49994   1970-03-05           1970
49995   2014-10-13           2014
49997   2014-10-27           2014
49998   2022-10-07           2022
49999   2018-06-22           2018

[40070 rows x 2 columns]


In [13]:
from collections import defaultdict

In [14]:
# Initialize an empty defaultdict to store the unique genres
genre_dict = defaultdict(int)

In [15]:
# Iterate over each row in the "Song genres" column
for genres in df_cleaned['Song genres'].dropna():  # Drop NaNs to avoid errors
    # Split the genres by commas (if multiple genres are listed)
    genre_list = genres.split(',')
    
    # Iterate over the list of genres
    for genre in genre_list:
        # Strip any leading or trailing spaces and add the genre to the dictionary
        genre_dict[genre.strip().lower()] += 1

# Convert the dictionary to a regular dictionary to view the result (optional)
genre_dict = dict(genre_dict)

# Display the dictionary with unique genres and their count
print(genre_dict)

{'r&b': 4655, 'pop': 16550, 'folk': 1303, 'alternative': 5151, 'rock': 12848, 'electro': 5078, 'hip hop': 9391, 'soundtrack': 1983, 'metal': 1159, 'country': 1636, 'latin': 6256, 'asian': 1263, 'jazz': 986, 'blues': 303, 'others': 1005, 'reggae': 376, 'african': 51, 'kids': 249, 'classical': 459, 'spirituals': 231, 'sports': 11, 'holiday': 10, 'mena': 11, 'european': 18, 'instrumental': 2, 'spoken': 57}


In [16]:
# Sort the dictionary by the count (values) in descending order
sorted_genres = {k: v for k, v in sorted(genre_dict.items(), key=lambda item: item[1], reverse=True)}

# Display the sorted dictionary
print(sorted_genres)

{'pop': 16550, 'rock': 12848, 'hip hop': 9391, 'latin': 6256, 'alternative': 5151, 'electro': 5078, 'r&b': 4655, 'soundtrack': 1983, 'country': 1636, 'folk': 1303, 'asian': 1263, 'metal': 1159, 'others': 1005, 'jazz': 986, 'classical': 459, 'reggae': 376, 'blues': 303, 'kids': 249, 'spirituals': 231, 'spoken': 57, 'african': 51, 'european': 18, 'sports': 11, 'mena': 11, 'holiday': 10, 'instrumental': 2}


In [17]:
# Get the top 15 genres
top_15_genres = dict(list(sorted_genres.items())[:15])

# Print the top 15 genres
print(top_15_genres)

{'pop': 16550, 'rock': 12848, 'hip hop': 9391, 'latin': 6256, 'alternative': 5151, 'electro': 5078, 'r&b': 4655, 'soundtrack': 1983, 'country': 1636, 'folk': 1303, 'asian': 1263, 'metal': 1159, 'others': 1005, 'jazz': 986, 'classical': 459}


In [18]:
# Get the count of non-NaN values in the 'Spotify stream Total' column
count_non_nan_values = filtered_df['Spotify stream Total'].count()

# Display the count
print(count_non_nan_values)

50000


In [19]:
# Count non-NaN values in the 'Spotify stream Total' column that are not 0
count_non_nan_non_zero = filtered_df[filtered_df['Spotify stream Total'] != 0]['Spotify stream Total'].count()

# Display the count
print(count_non_nan_non_zero)

50000


In [20]:
from tabulate import tabulate

# Pretty print the first 20 rows of the DataFrame using tabulate
print(tabulate(df_cleaned.head(20), headers='keys', tablefmt='pretty', showindex=False))

+-----------------------------------------------+--------------------------------------+------------------------------------------------------------------------------------+--------------+----------------+--------------+----------------------------+-----------------------------------------------+--------------------------------------+-----------------------------------------------------------------+----------------------+-----------------------+-------------------------+--------------------------+---------------------------+-----------------------------+---------------------+---------------------+---------------------+---------------------+---------------+
|                     Song                      |              Song uuid               |                                      Song url                                      |  Song isrc   |  ISRC Country  | Release date |        Song genres         |                   Artist(s)                   |            Artist Country            